In [1]:
# Import
from dotenv import load_dotenv
import os
import sys
import requests
from requests import post
import json
import time
import pandas as pd
import ast

In [2]:
load_dotenv()

client_id_igdb = os.getenv("IGDB_ID_CLIENT")
client_secret_igdb = os.getenv("IGDB_SECRET_CLIENT")

In [3]:
print(client_id_igdb)
print(client_secret_igdb)

4pmce94am1hizog6ff9feucpmtki1q
t0bfpbrfjsxb4eqpt2kezg82p3uhsw


In [4]:
# Récupération du token d'accès pour IGDB

url = f"https://id.twitch.tv/oauth2/token?client_id={client_id_igdb}&client_secret={client_secret_igdb}&grant_type=client_credentials"

print(url)
response = requests.post(url)
if response.status_code == 200:
    response_json = response.json()
    if 'expires_in' in response_json == 0:
        print(f"Le token a expiré, il faut le renouveler")
    access_token = response_json['access_token']
    print(f"Access Token: {access_token}")
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)

https://id.twitch.tv/oauth2/token?client_id=4pmce94am1hizog6ff9feucpmtki1q&client_secret=t0bfpbrfjsxb4eqpt2kezg82p3uhsw&grant_type=client_credentials
Access Token: b3qcss3wxbj8al0okp17ltdk7up3iu


In [5]:
# Récuperer le nombres de jeux
url_igdb_engines_count = "https://api.igdb.com/v4/games/count"

headers = {
    'Client-ID': client_id_igdb,   
    'Authorization': f'Bearer {access_token}',
    }

response = post(url_igdb_engines_count, headers=headers, data='fields *;')
print(response.json())

{'count': 323447}


Recup Game_genre

In [8]:
# Fonction pour récupérer les type de support pour les langues
def get_all_game_genre():
    """
    Récupère les genre_id partie Game de l'API IGDB avec pagination.
    
    Returns:
        pd.DataFrame: DataFrame contenant toutes les perspectives de joueur.
    """
    all_game_genre = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/games', headers=headers, data = f'fields genres; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            all_game_genre.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(all_game_genre)

In [19]:
# Exporter le DataFrame en CSV
all_game_genre.to_csv('../csv/game_genre.csv', index=False)

Recup genre ID

In [21]:
# Fonction pour récupérer les type de support pour les langues
def get_all_genre_id():
    """
    Récupère les genre_id partie Game de l'API IGDB avec pagination.
    
    Returns:
        pd.DataFrame: DataFrame contenant toutes les perspectives de joueur.
    """
    all_genre = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/genre', headers=headers, data = f'fields name; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            all_genre.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(all_genre)

In [12]:
all_genre = get_all_genre_id()
display(all_genre)

,id,name
0,330684,Nightmare Kart: The Old Karts
1,177310,The Undying Beast
2,129194,Arcade Archives: XX Mission
3,282079,Rockman X: New Year 2023
4,63844,Ace wo Nerae!
...,...,...
322132,89219,Stronghold Crusader II: The Templar &The Duke
322133,8993,Anomaly Defenders
322134,154420,Catacombs Pack
322135,11261,The Black Mirror


In [20]:
# Exporter le DataFrame en CSV
all_genre.to_csv('../csv/genre_id.csv', index=False)

PLATFORM_VERSION

In [6]:
# Fonction pour récupérer les id + version_name
def get_platform_version():
    """
    Récupère les id des platform et le nom de la version de l'API IGDB.
    
    Returns:
        pd.DataFrame: DataFrame contenant toutes les perspectives de joueur.
    """
    platform_version = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/platform_versions', headers=headers, data = f'fields name; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            platform_version.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(platform_version)

In [ ]:
df_platform_version = get_platform_version()

In [10]:
df_platform_version.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv/plaform_version.csv', index=False)

PLATFORM_PLATFORM_VERSION

In [11]:
# Fonction pour récupérer les platform id + version id
def PLATFORM_PLATFORM_VERSION():
    """
    Récupère les platfor_id et version_id 
    
    Returns:
        pd.DataFrame: DataFrame contenant toutes les perspectives de joueur.
    """
    platform_platform_version = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/platforms', headers=headers, data = f'fields versions; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            platform_platform_version.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(platform_platform_version)

In [16]:
df_platform_platform_version = PLATFORM_PLATFORM_VERSION()
df_platform_platform_version = df_platform_platform_version.explode('versions')
df_platform_platform_version

,id,versions
0,306,423
1,339,456
2,376,494
3,12,2
3,12,3
...,...,...
218,130,282
218,130,173
218,130,503
219,159,228


In [17]:
df_platform_platform_version.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv/plaform_platform_version.csv', index=False)